# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

<ipython-input-1-1499ab5789c7>:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter
import timm
import torch_optimizer
#from torchvision.models import wide_resnet50_2, resnet18
#from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})

# original library
import utils
import common as com
import pytorch_modeler as modeler
from pytorch_model import EfficientNet_b1 as Model
from pytorch_model import EWCLoss
#from pytorch_utils import filtered_load_model
#import models

import librosa
import IPython
import librosa.display

FISHER_DIR = config['IO_OPTION']['FISHER_ROOT']
fisher_path = f'{FISHER_DIR}/{machine_types[0]}_fisher.pkl'
fisher = pd.read_pickle(fisher_path)

## load config and set logger

In [4]:
log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

## Setting

In [5]:
# Setting seed
modeler.set_seed(42)

In [6]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_dir = INPUT_ROOT + "/dev_data"
add_dev_dir = INPUT_ROOT + "/add_dev_data"
# machine type
machine_types = os.listdir(dev_dir)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
PREMODEL_ROOT = config['IO_OPTION']['PREMODEL_ROOT']

TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
OUT_FEATURE_DIR = OUTPUT_ROOT + '/extraction_features'
PRED_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/pred'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(OUT_FEATURE_DIR, exist_ok=True)
os.makedirs(PRED_DIR, exist_ok=True)

# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/HDD1TB/research/DCASE_experiments/domain_adaptation/exp2/PANDA/config.yaml'

In [7]:
torch.cuda.is_available()

True

## make path list

- __train_paths__ ([dict]): Each Machine Type
    - __ToyCar__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) :paths
    - __ToyTrain__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) : paths
    - ...

In [8]:
dev_paths = {}

for machine_type in machine_types:
    # dev train
    dev_train_paths = [f"{dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/train")]
    dev_train_paths = sorted(dev_train_paths)
    
    print(f'Machine {machine_type}', end=',')
    print(f'dev_train_size : {len(dev_train_paths)}', end=', ')
    dev_train_source_paths = [path for path in dev_train_paths if 'source' in path]
    print(f'source_size : {len(dev_train_source_paths)}', end=', ')
    dev_train_target_paths = [path for path in dev_train_paths if 'target' in path]
    print(f'target_size : {len(dev_train_target_paths)}', end=', ')
    # source mode
    dev_train_paths = dev_train_source_paths + dev_train_target_paths
    
    # add_dev train
    add_dev_paths = [f"{add_dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{add_dev_dir}/{machine_type}/train")]
    add_dev_paths = sorted(add_dev_paths)
    print(f'adddev_train_size : {len(dev_train_paths)}', end=', ')
    add_dev_source_paths = [path for path in add_dev_paths if 'source' in path]
    print(f'source_size : {len(add_dev_source_paths)}', end=', ')
    add_dev_target_paths = [path for path in add_dev_paths if 'target' in path]
    print(f'target_size : {len(add_dev_target_paths)}')
    # source mode
    add_dev_paths = add_dev_source_paths + add_dev_target_paths
    print('==============================================================')
    
    # dev_source valid
    dev_source_paths = [f"{dev_dir}/{machine_type}/source_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/source_test")]
    dev_source_paths = sorted(dev_source_paths)
    
    # dev_target valid
    dev_target_paths = [f"{dev_dir}/{machine_type}/target_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/target_test")]
    dev_target_paths = sorted(dev_target_paths)
    
    # bundle
    dev_paths[machine_type] = {}
    dev_paths[machine_type]['train'] = dev_train_paths + add_dev_paths
    dev_paths[machine_type]['valid_source'] = dev_source_paths
    dev_paths[machine_type]['valid_target'] = dev_target_paths

Machine fan,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine gearbox,dev_train_size : 3026, source_size : 3017, target_size : 9, adddev_train_size : 3026, source_size : 3105, target_size : 9
Machine pump,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine slider,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine ToyCar,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine ToyTrain,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine valve,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9


In [9]:
machine_types

['fan', 'gearbox', 'pump', 'slider', 'ToyCar', 'ToyTrain', 'valve']

In [10]:
from copy import deepcopy

In [11]:
#############################################################################
# run
#############################################################################
def run(machine_type, dev_paths):
    com.tic()
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dev_train_paths
    dataloaders_dict = modeler.make_dataloader(dev_paths, machine_type)
    # parameter setting
    logger.info('TRAINING')
    model = Model(num_center = 1)
    # load weight
    premodel_path = f'{PREMODEL_ROOT}/{machine_type}_model.pkl'
    pretrained_dict = torch.load(premodel_path)
    model.load_state_dict(pretrained_dict, strict=False)
    # load fisher
    FISHER_DIR = config['IO_OPTION']['FISHER_ROOT']
    fisher_path = f'{FISHER_DIR}/{machine_type}_fisher.pkl'
    fisher = pd.read_pickle(fisher_path)
    # frozen_model
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    frozen_model = deepcopy(model).to(device)
    frozen_model.eval()
    utils.freeze_model(frozen_model)
    # set ewc loss
    ewc_criterion = EWCLoss(frozen_model, fisher)
    
    # partially freeze
    utils.freeze_parameters(model)
    
    optimizer = torch.optim.Adam(model.parameters(),lr=1e-4)
    output_dict, model, pred_df = modeler.run_training(model, dataloaders_dict, writer, optimizer, ewc_criterion)
    
    # output
    feature_out_path = f'{OUT_FEATURE_DIR}/{machine_type}_features.pkl'
    model_out_path = f'{MODEL_DIR}/{machine_type}_model.pkl'
    pred_out_path = f'{PRED_DIR}/{machine_type}_pred.csv'
    # save
    pd.to_pickle(output_dicts, feature_out_path)
    torch.save(model.state_dict(), model_out_path)
    score_df.to_csv(pred_out_path)
    logger.info(f'SAVE SUCCESS : {model_out_path}')
    
    com.toc()

# run

In [12]:
for machine_type in machine_types:
    run(machine_type, dev_paths)

2021-09-28 11:10:28,729 - 00_train.py - INFO - TARGET MACHINE_TYPE: fan
2021-09-28 11:10:28,730 - 00_train.py - INFO - MAKE DATA_LOADER
2021-09-28 11:10:28,733 - 00_train.py - INFO - TRAINING
  0%|          | 0/24 [00:00<?, ?it/s]

use: cuda:0


100%|██████████| 3/3 [00:01<00:00,  1.51it/s]
2021-09-28 11:10:50,248 - pytorch_modeler.py - INFO - epoch:1/50, tr_loss:27.446152, src_loss:3.145747, src_mean_auc:0.662183, tgt_loss:3.111046, tgt_mean_auc:0.629583,


,AUC,pAUC
Source_0,0.550650,0.530000
Source_1,0.745400,0.688421
Source_2,0.690500,0.660526
Target_0,0.530600,0.515263
Target_1,0.710000,0.577368
Target_2,0.648150,0.588421
mean,0.645883,0.593333
h_mean,0.635473,0.586765


100%|██████████| 3/3 [00:02<00:00,  1.37it/s]
2021-09-28 11:11:08,637 - pytorch_modeler.py - INFO - epoch:2/50, tr_loss:23.451335, src_loss:2.887184, src_mean_auc:0.670617, tgt_loss:2.882750, tgt_mean_auc:0.621933,


...

,AUC,pAUC
Source_0,0.603300,0.495263
Source_1,0.742800,0.654737
Source_2,0.751950,0.674737
Target_0,0.533950,0.501053
Target_1,0.618600,0.551053
Target_2,0.692400,0.586316
mean,0.657167,0.577193
h_mean,0.647436,0.569012


100%|██████████| 3/3 [00:02<00:00,  1.48it/s]
2021-09-28 11:14:44,461 - pytorch_modeler.py - INFO - epoch:13/50, tr_loss:11.738807, src_loss:1.417742, src_mean_auc:0.685750, tgt_loss:1.421839, tgt_mean_auc:0.578900,


,AUC,pAUC
Source_0,0.561700,0.498947
Source_1,0.743600,0.649474
Source_2,0.751950,0.646842
Target_0,0.555500,0.506316
Target_1,0.548200,0.530000
Target_2,0.633000,0.538421
mean,0.632325,0.561667
h_mean,0.621176,0.555092


 42%|████▏     | 10/24 [00:06<00:09,  1.49it/s]


KeyboardInterrupt: 

In [ ]:
#run(machine_types[1], dev_paths)

In [ ]:
feature_out_path = f'{OUT_FEATURE_DIR}/{machine_types[0]}_features.pkl'
tst = pd.read_pickle(feature_out_path)

In [ ]:
tst['val_tgt']

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(tst['val_src']['feature'].astype(float), aspect='auto')

In [ ]:
tst['val_src']['feature'].shape

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(tst['val_tgt']['feature'].astype(float), aspect='auto')